<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Привет! Поздравляю тебя с первым твоим проектом и спасибо, что сдал задание:) Ты проделал большую работу. Далее в файле мои коммефнтарии ты сможешь найти в ячейках, аналогичных данной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта. 

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

### Вывод

In [1]:
import pandas as pd
credit_data = pd.read_csv('/datasets/data.csv')
#print(credit_data.head(15))
print(credit_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Проведя первичный анализ данных, можно сделать следующие выводы: 1. В столбце со сроком занятости присутсвуют  отрицательные значения и аномально большие. 2. В столбце "образование" отсуствует единая стилистика написания информации, что может повлиять на корректность анализа. 3. В столбце "Назначение кредита" одинаковые цели называются по-разному: "недвижимость, покупка жилья, операции с жильём" и т.д.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошее начало, так же, для большего понимания данных можно было бы использовать метод `describe`


### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
def exp_in_days(row): #Создаем функцию для избавления от отрицательных значений
    exp = row['days_employed']
    if exp > 0:
        return exp #оставляем положительные значения
    else:
        exp = abs(exp)  #избавляемся от отрицательных значений
        return exp
 
   
credit_data['days_employed'] = credit_data.apply(exp_in_days, axis=1) 
income_med = credit_data['total_income'].median()
credit_data['total_income'] = credit_data['total_income'].fillna(value = income_med)
employed_med = credit_data['days_employed'].median()
credit_data['days_employed'] = credit_data['days_employed'].fillna(value = employed_med)
print(credit_data.head(25))


    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

### Вывод

Проведя анализ пропущенных значений в таблице, что в 2174 строках отсуствовали данные по стажу и по доходу одновременно. Для заполнения пропущенных значений была использована медиана, чтобы минимизировать влияние аномальных значений.

<div style="border:solid orange 2px; padding: 20px"> <h1 style="color:orange; margin-bottom:20px">Комментарий наставника</h1>

Тут лучше не заполнять пропуски просто медианой, средним или нулём, требуется более обоснованное заполнение пропусков исходя из имеющихся данных



### Замена типа данных

In [3]:
credit_data['days_employed'] = credit_data['days_employed'].astype('int') #меняем тип данных на целочисленный
credit_data['total_income'] = credit_data['total_income'].astype('int')#меняем тип данных на целочисленный
print(credit_data.info()) # проверяем, что тип данных в столбце заменился

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

Для наглядности и удобства использования данных меняем тип данных столбцов со стажем и доходом с float на int используя метод astype(). 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Замена типа данных выполнена верно

### Обработка дубликатов

In [4]:
#print(credit_data['education'].value_counts())  #Проверяем значения столбца education
#print(credit_data['income_type'].value_counts())  #Проверяем значения столбца income_type
#print(credit_data['gender'].value_counts())  #Проверяем значения столбца gender 
#print(credit_data['family_status'].value_counts())  #Проверяем значения столбца family_status
#print(credit_data['family_status'].value_counts())  #Проверяем значения столбца family_status
credit_data['education'] = credit_data['education'].str.lower() #Приводим значения в столбце education к нижнему регистру


print ('Количество дубликатов в данных:', credit_data.duplicated().sum())



Количество дубликатов в данных: 71


### Вывод

Произведена замена регистра в столбце образования методом str.lower(), поскольку ввиду ручного ввода данных регистр не унифицирован, что может повлиять на корректность анализа. Проверяем замену методом value_counts. Проверяем наличие дубликатов. Дубликатов не обнаружено.

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Стоило так же обработать(удалить) найденный дубликаты

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
for i in range (len(credit_data)):
    lemmas = m.lemmatize(credit_data['purpose'][i]) # приводим данные из стобца 'purpose' к лемме

credit_data['purpose_lemma'] = credit_data['purpose'].apply(m.lemmatize) # создаем новый столбец с лемматизированными данными
print(credit_data['purpose_lemma']) # проверяем значения нового столбца



0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: purpose_lemma, Length: 21525, dtype: object


### Вывод

Мы привели разные названия одних и тех же по сути категорий к более-менее унифицированной форме, к которой удобно обращаться и с которой можно проводить категоризацию данных.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>


Лемматизация проведена верно

### Категоризация данных

In [6]:
#print(credit_data['children'].value_counts()) проверяем значение столбца 'children'
# Видим выбросы данных в значении 20 и -1. Отрицательное значение детей быть не может, 20 явно является ошибчным, 
#поскольку значений от 5 до 19 детей не встречается вовсе. Поскольку аномальных значений ничтожно мало,
# для категоризации мы их проигнорируем.
# Категоризацию данных построим по принципу зависимости количества детей от возможной просрочки платежа
# Для понимания зависимости объеденим в категорию следующие значения:
# "Многодетными должниками' будут признаны в значении: 3 <= children <= 5 и значении debt = 1
# "Должниками с детьми" будут признаны в значении: 1 <= children < 3 и значении debt = 1
# "Должниками без детей" будут признаны в значении: children = 0 и значении debt = 1
# 'Плательщики' в значении:0 <= children <= 5  и значении debt = 0
# "Многодетными плательщиками' будут признаны в значении: 3 <= children <= 5 и значении debt = 0
# "Плательщики с детьми" будут признаны в значении: 1 <= children < 3 и значении debt = 0
# "Плательщиками без детей" будут признаны в значении: children = 0 и значении debt = 0


def children_and_debt(row):
    children = row['children']
    debt = row['debt']
   
    if 3 <= children <= 5:
        if debt == 1:
            return 'Многодетный должник'
       
    if 1 <= children < 3:
        if debt == 1:
            return 'Должник с детьми'
        
    if children == 0:
        if debt == 1:
            return 'Должник без детей'
        
    if 3 <= children <= 5:
        if debt == 0:
            return 'Многодетный плательщик'
       
    if 1 <= children < 3:
        if debt == 0:
            return 'Плательщик с детьми'
        
    if children == 0:
        if debt == 0:
            return 'Плательщик без детей'
        

credit_data['children_return'] = credit_data.apply(children_and_debt, axis=1) #добавляем новый столбец со значениями после 
#применения функции 'children_return'

#Проведем категоризацию данных по семейному положению, для этого
# сопоставим семейное положение и их ID параметром:
# женат / замужем -  0;   гражданский брак -1;    довец / вдова - 2;
# в разводе - 3;          не женат / не замужем - 4  

def family_status_and_debt(row):
    family_status = row['family_status_id']
    debt = row['debt']
  
    if family_status == 0:
        if debt == 1:
            return 'Должник в браке'
       
    if family_status == 0:
        if debt == 0:
            return 'Плательщик  в браке'
        
        
    if family_status == 1:
        if debt == 1:
            return 'Должник в гражданском браке'
    if family_status == 1:
        if debt == 0:
            return 'Плательщик в гражданском браке'
        
      
    if family_status == 2:
        if debt == 1:
            return 'Должник вдовец / вдова'
       
    if family_status == 2:
        if debt == 0:
            return 'Плательщик вдовец / вдова'
        
      
    if family_status == 3:
        if debt == 1:
            return 'Должник в разводе'
       
    if family_status == 3:
        if debt == 0:
            return 'Плательщик в разводе'    
    if family_status == 4:
        if debt == 1:
            return 'Должник не женат / не замужем'
       
    if family_status == 4:
        if debt == 0:
            return 'Плательщик не женат / не замужем'
credit_data['family_status_and_debt'] = credit_data.apply(family_status_and_debt, axis=1) #добавляем новый столбец со 
#значениями после применения функции 'family_status_and_debt'

#Категоризируем данные по достатку клиентов
# Для этого разобьем клиентов на три условные группы - с доходом до 50 000 - низкий достаток
# от 50 000  до 150 000 рублей - средний достаток, cвыше 150 000 - высокий достаток
def total_income_and_debt(row):
    total_income = row['total_income']
    debt = row['debt']
  
    if 1 < total_income <= 50000:
        if debt == 1:
            return 'Должник с низким достатком'
       
    if 1 < total_income <= 50000:
        if debt == 0:
            return 'Плательщик с низким достатком'
        
     
    if 50000  < total_income <= 150000:
        if debt == 1:
            return 'Должник со средним достатком'
       
    if 50000  < total_income <= 150000:
        if debt == 0:
            return 'Плательщик со средним достатком'
        
       
    if total_income > 150000:
        if debt == 1:
            return 'Должник с высоким достатком'
       
    if total_income > 150000:
        if debt == 0:
            return 'Плательщик с высоким достатком'
    

 
credit_data['total_income_and_debt'] = credit_data.apply(total_income_and_debt, axis=1) #добавляем новый столбец с данными 
#после примения функции 'total_income_and_debt'






### Вывод

Мы категоризировали данные по коичеству детей, семейному положению и достатку. Теперь их можно использовать для ответы на вопросыю

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Категоризация проведена верно

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:

print(credit_data['children_return'].value_counts())

print ('Процент должников среди клиентов без детей: {:.1%}'.format(1063 / 13086))
print ('Процент должников среди клиентов с детьми: {:.1%}'.format(638 / 6235))
print ('Процент должников среди многодетных клиентов: {:.1%}'.format(31 / 349))

Плательщик без детей      13086
Плательщик с детьми        6235
Должник без детей          1063
Должник с детьми            638
Многодетный плательщик      349
Многодетный должник          31
Name: children_return, dtype: int64
Процент должников среди клиентов без детей: 8.1%
Процент должников среди клиентов с детьми: 10.2%
Процент должников среди многодетных клиентов: 8.9%


### Вывод

Как видно из этой группировки, процент должников среди клиентов у которых 1-3 ребенка составляет 10.2%, что ялвяется максимальным результатом для всех категорий. На втором месте идут многодетные клиенты, у которых 3-5 (включительно) детей - 8.9% должников. Самыми дисциплинированными в рамках возврата кредита являются бездетные клиенты - 8.1% должников.

In [8]:
print (credit_data['family_status_and_debt'].value_counts())
print ('Процент должников среди клиентов в браке: {:.1%}'.format(931 / 11449))
print ('Процент должников среди клиентов в гражданском браке: {:.1%}'.format(388 / 3789))
print ('Процент должников среди клиентов не в браке: {:.1%}'.format(274 / 2539))
print ('Процент должников среди клиентов в разводе: {:.1%}'.format(85 / 1110))
print ('Процент должников среди вдовствующих клиентов: {:.1%}'.format(63 / 897))

Плательщик  в браке                 11449
Плательщик в гражданском браке       3789
Плательщик не женат / не замужем     2539
Плательщик в разводе                 1110
Должник в браке                       931
Плательщик вдовец / вдова             897
Должник в гражданском браке           388
Должник не женат / не замужем         274
Должник в разводе                      85
Должник вдовец / вдова                 63
Name: family_status_and_debt, dtype: int64
Процент должников среди клиентов в браке: 8.1%
Процент должников среди клиентов в гражданском браке: 10.2%
Процент должников среди клиентов не в браке: 10.8%
Процент должников среди клиентов в разводе: 7.7%
Процент должников среди вдовствующих клиентов: 7.0%


### Вывод

Как видно из представленной группировки семейное положение играет большую роль при возрвате кредита: так, люди в браке более дисциплинированны в возврате кредита процент должников - 8.1%, в то время как холостые клиенты допускают просрчоки, процент должников - 10.8%. Самыми дисциплинированными являются вдовствующие клиенты(7.0%) и клиенты в разводе(7.7%).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:

print(credit_data['total_income_and_debt'].value_counts()) 
print ('Процент должников среди клиентов с высоким достатком: {:.1%}'.format(726 / 8458))
print ('Процент должников среди клиентов со средним достатком: {:.1%}'.format(992 / 10977))
print ('Процент должников среди клиентов с низким достатком: {:.1%}'.format(23 / 349))



Плательщик со средним достатком    10977
Плательщик с высоким достатком      8458
Должник со средним достатком         992
Должник с высоким достатком          726
Плательщик с низким достатком        349
Должник с низким достатком            23
Name: total_income_and_debt, dtype: int64
Процент должников среди клиентов с высоким достатком: 8.6%
Процент должников среди клиентов со средним достатком: 9.0%
Процент должников среди клиентов с низким достатком: 6.6%


### Вывод

Анализ зависимости дохода от риска просрочек показал, что клиенты с высоким и средним достатком склонны более допускать просрочки, чем клиенты с низким достатком.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:

#print(credit_data['purpose_lemma'].value_counts()) #проверяем значения столбца 'purpose_lemma'
def purpose_lemma(row): #создаем функцию для распределния на 4 группы: 1 - недвижимость, 2 - автомобиль, 3- образование, 4 - свадьба
    if 'жилье' in row:
        return 1
    elif 'недвижимость' in row: 
        return 1

    elif 'автомобиль' in row:
        return 2
    elif 'образование' in row:
        return 3
    elif 'свадьба' in row:
        return 4

    
credit_data['purpose_lemma_id'] = credit_data['purpose_lemma'].apply(purpose_lemma)


credit_data_pivot = credit_data.pivot_table(index='purpose_lemma_id', columns='debt', values='purpose', aggfunc='count')#создаем сводную таблицу по числу должников среди категорий
print(credit_data_pivot)
print ('Процент должников среди клиентов с кредитом на недвижимость: {:.1%}'.format(782 / 10058))
print ('Процент должников среди клиентов с кредитом на автомобиль: {:.1%}'.format(403 / 3912))
print ('Процент должников среди клиентов кредитом на образование: {:.1%}'.format(370 / 3652))
print ('Процент должников среди клиентов кредитом на свадьбу: {:.1%}'.format(186 / 2162))

debt                  0    1
purpose_lemma_id            
1                 10058  782
2                  3912  403
3                  3652  370
4                  2162  186
Процент должников среди клиентов с кредитом на недвижимость: 7.8%
Процент должников среди клиентов с кредитом на автомобиль: 10.3%
Процент должников среди клиентов кредитом на образование: 10.1%
Процент должников среди клиентов кредитом на свадьбу: 8.6%


### Вывод

Наименее дисциплинированными являются клиенты с кредитами на образование и автомобиль. Самый "возвращаемый" кредит - на недвижимость.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Все выводы верны, код написан правильно, радует, что используешь сводные таблицы)


### Шаг 4. Общий вывод

Подведём итог: 
 1. Наличие детей - клиенты без детей лучше овзвращают кредиты, чем клиенты с детьми.
 2. Семейное положение - холостые клиенты и клиенты в гражданском браке хуже возвращают кредиты, чем клиенты в браке, разводе или вдовствующие.
 3. Клиенты с низким достатком совершают меньше просрочек, чем клиенты со средним и высоким достатком.
 4. Кредиты на свадьбу и жилье возвращаются заметно лучше, чем кредиты на образование и автомобиль.
 Таким образом мы выяснили, что все 4 фактора оказывают зависимость к возврату долга в банке. 
 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хороший вывод, как совет на будущее - можешь в выводе писать рекомендации по использованию данного анализа


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции. В качестве совета предлагаю глубже изучить и начать чаще применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем, а так же изучить средства для построения изображений в Python для более глубокого понимания данных.
#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)


<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>
Работа принята!) Очень рад, что у тебя всё получилось и надеюсь тебе понравится весь курс, будь усерден и у тебя всё получится! Удачи в будущих проектах)